In [1]:
from repsheet_backend.genai import generate_text
from repsheet_backend.bills import get_latest_bill_text_path, get_every_bill_voted_on_by_a_member
from repsheet_backend.common import db_connect, BILLS_TABLE
from typing import Optional

In [2]:
with open("prompts/summarize-bill/001.txt", "r") as f:
    prompt_template = f.read()

def summarise_bill(bill) -> Optional[str]:
    xml_path = get_latest_bill_text_path(bill)
    if xml_path is None:
        return None
    with open(xml_path, "r") as f:
        xml_text = f.read()
    prompt = prompt_template + xml_text
    return generate_text(prompt)

In [3]:
bill_summaries = []

for bill in get_every_bill_voted_on_by_a_member()[:5]:
    summary = summarise_bill(bill)
    if summary is not None:
        bill_summaries.append({
            "bill": bill,
            "summary": summary
        })

In [ ]:
with db_connect() as db:
    # Clear the table before inserting new summaries
    db.execute(f"DELETE FROM {BILLS_TABLE}")

    insert_values = []
    for bs in bill_summaries:
        bill = bs["bill"]
        summary = bs["summary"]
        summary = summary.removeprefix("```json\n").removesuffix("\n```")
        insert_values.append((str(bill), summary))

    # Insert the new summaries
    db.executemany(f"INSERT INTO {BILLS_TABLE} ([Bill ID], Summary) VALUES (?, ?)", insert_values)
